# Tradução dos Códigos de Ocupação O*NET para ocupações do Brasil (COD 2010)

Para aplicar a metodologia de Dingel e Neiman, é necessário realizar a tradução dos códigos de ocupações para ocupações do Brasil.

**Passo-a-passo da tradução dos códigos de ocupação, segundo artigo da FGV:**

1. Primeiro, realizamos uma tradução das 968 ocupações da O*NET para a Standard Occupational Classification de 2010 (SOC-2010), utilizada na pesquisa por domicílios americana (Current Population Survey - CPS);
2. Em seguida, reclassificamos as ocupações da SOC-2010 para a ISCO-08, para então; 
3. traduzirmos para a COD utilizada na PNAD Contínua. 

As reclassificações foram realizadas conforme o diagrama abaixo.

#### Diagrama para tradução dos códigos de ocupação (FGV, 2021)
![Diagrama](../Anexos/diagrama_traducao.png)

Link para base de ocupações da O*NET: [O*NET OnLine](https://www.onetonline.org/find/all)

Link para base de ocupações da SOC-2010: [U.S. BUREAU OF LABOR STATISTICS](https://www.bls.gov/soc/2010/home.htm)

Link para base de ocupações da ISCO-08: [International Labour Organization](https://isco-ilo.netlify.app/en/isco-08/)

Repositório do estudo Work at Home (Dingel e Neiman): [Work at Home](https://github.com/jdingel)

Arquivo CSV com as ocupações teletrabalháveis: [occupations_workathome.csv](https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv)



In [1]:
#pacotes
import pandas as pd
import numpy as np

In [2]:
#Leitura das bases (tabelas de ocupação e de pessoas)

onet = pd.read_excel('../Anexos/Occupations_ONET.xlsx') #Ocupações O*NET
soc2010 = pd.read_excel('../Anexos/Occupations_SOC-2010.xlsx') #Ocupações SOC-2010
isco08 = pd.read_excel('../Anexos/Occupations_ISCO-08.xlsx') #Ocupações ISCO-08
#cod2010 = pd.read_excel('../Anexos/COD 2010.xlsx', ) #Ocupações COD 2010

cod2010 = pd.read_excel('../Anexos/COD 2010.xlsx', dtype={'Cod': str}) #Ocupações COD 2010

isco_soc_reclassificado = pd.read_excel('../Anexos/soc2010-ISCO08-crosswalk.xlsx', dtype={'ISCO-08 Code':str})

onet_teleworkable = pd.read_csv('https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv')

In [3]:
isco_soc_reclassificado['major_isco'] = isco_soc_reclassificado['ISCO-08 Code'].apply(lambda x: x[:1])

In [4]:
#Ocupações na base O*NET que podem ser realizadas em teletrabalho
onet_teleworkable.head()

,onetsoccode,title,teleworkable
0,11-1011.00,Chief Executives,1
1,11-1011.03,Chief Sustainability Officers,1
2,11-1021.00,General and Operations Managers,1
3,11-2011.00,Advertising and Promotions Managers,1
4,11-2021.00,Marketing Managers,1


### Tradução do O\*NET (filtrado por Dingel e Neiman) para o SOC-2010 

A tradução deve ser feita usando como refência o código do Major Group e título da ocupação

In [5]:
soc2010filtrado = soc2010[['Detailed Occupation', 'Title']]
soc2010filtrado.columns = ['code_SOC', 'Title_SOC']
soc2010filtrado.dropna(inplace=True)

soc2010['code_SOC'] = soc2010.iloc[:, :4].apply(lambda x: ''.join(x.dropna().astype(str)), axis=1)
#Captura somente o codigo major
soc2010filtrado['MAJOR_SOC'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])

/tmp/ipykernel_14802/1001432072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc2010filtrado.dropna(inplace=True)
/tmp/ipykernel_14802/1001432072.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc2010filtrado['MAJOR_SOC'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])


In [6]:
onet_filtrado = onet_teleworkable.copy()
onet_filtrado.columns = ['code_ONET', 'Title_ONET', 'teleworkable']

#captura somente o codigo major
onet_filtrado['code_ONET'] = onet_filtrado['code_ONET'].apply(lambda x: x[0:-3])

In [7]:
#join onet_filtrado e soc2010filtrado by column Code_ONET and code_SOC

onet_soc = pd.merge(onet_filtrado, isco_soc_reclassificado, how='left', left_on='code_ONET', right_on='2010 SOC Code')

Até aqui, ONET e SOC-2010 foram agregados com sucesso. Agora, vamos fazer o mesmo com ISCO-08 e COD-2010.

### Tradução de ISC08 para COD 2010

In [36]:
cod2010.head(2)

,Cod,Grandes Grupos,Subgrupos Principal,Subgrupos,Grupos de Base,Titulação
0,1,1,NaN,NaN,NaN,DIRETORES E GERENTES
1,11,1,11.0,NaN,NaN,"DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRA..."


In [37]:
cod2010_filtrado = cod2010[['Cod', 'Subgrupos Principal', 'Titulação']]
cod2010_filtrado.columns = ['code_COD','Subgrupo', 'Title_COD']

cod2010_filtrado['code_COD'] = cod2010_filtrado['code_COD'].astype(str)

/tmp/ipykernel_14802/766749561.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cod2010_filtrado['code_COD'] = cod2010_filtrado['code_COD'].astype(str)


In [38]:
onet_soc['major_isco'].fillna(0, inplace=True)

In [39]:
isco08_filtrado = isco08[['ISCO 08 Code', 'Major', 'Title EN']]
isco08_filtrado.columns = ['code_ISCO', 'major_ISCO', 'Title_ISCO']

isco08_filtrado['code_ISCO'] = isco08_filtrado['code_ISCO'].astype(str) 
isco08_filtrado['major_ISCO'] = isco08_filtrado['major_ISCO'].astype(str) 

/tmp/ipykernel_14802/127245518.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isco08_filtrado['code_ISCO'] = isco08_filtrado['code_ISCO'].astype(str)
/tmp/ipykernel_14802/127245518.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isco08_filtrado['major_ISCO'] = isco08_filtrado['major_ISCO'].astype(str)


In [40]:
isco_cod = pd.merge(isco08_filtrado, cod2010, how='left', left_on='major_isco', right_on='Cod')

Até aqui, temos ISCO-08 e COD 2010 agregados. Agora, vamos fazer o join com as ocupações agregadas.

In [19]:
isco_cod['Titulação'].fillna('OUTROS', inplace=True)
isco_cod['Cod'].fillna(0, inplace=True)

### Tradução de SOC-2010 para ISCO-08

In [20]:
onet_soc['ISCO-08 Code'] = onet_soc['ISCO-08 Code'].astype(str)

In [21]:
onet_soc['isco_major'] = onet_soc['ISCO-08 Code'].apply(lambda x: x[:1])

In [22]:
onet_soc['isco_major'].sort_values()

0       1
94      1
93      1
92      1
91      1
       ..
249     n
253     n
1221    n
801     n
524     n
Name: isco_major, Length: 1339, dtype: object

In [71]:
onet_soc

,code_ONET,Title_ONET,teleworkable,2010 SOC Code,2010 SOC Title,part,ISCO-08 Code,ISCO-08 Title EN,major_isco,isco_major
0,11-1011,Chief Executives,1,11-1011,Chief Executives,*,1112,Senior government officials,1,1
1,11-1011,Chief Executives,1,11-1011,Chief Executives,*,1113,Traditional chiefs and heads of villages,1,1
2,11-1011,Chief Executives,1,11-1011,Chief Executives,*,1120,Managing directors and chief executives,1,1
3,11-1011,Chief Sustainability Officers,1,11-1011,Chief Executives,*,1112,Senior government officials,1,1
4,11-1011,Chief Sustainability Officers,1,11-1011,Chief Executives,*,1113,Traditional chiefs and heads of villages,1,1
...,...,...,...,...,...,...,...,...,...,...
1334,53-7073,Wellhead Pumpers,0,53-7073,Wellhead Pumpers,NaN,8113,Well drillers and borers and related workers,8,8
1335,53-7081,Refuse and Recyclable Material Collectors,0,53-7081,Refuse and Recyclable Material Collectors,*,9611,Garbage and recycling collectors,9,9
1336,53-7081,Refuse and Recyclable Material Collectors,0,53-7081,Refuse and Recyclable Material Collectors,*,9612,Refuse sorters,9,9
1337,53-7111,Mine Shuttle Car Operators,0,53-7111,Mine Shuttle Car Operators,NaN,8311,Locomotive engine drivers,8,8


In [73]:
cod2010

,Cod,Grandes Grupos,Subgrupos Principal,Subgrupos,Grupos de Base,Titulação
0,1,1,NaN,NaN,NaN,DIRETORES E GERENTES
1,11,1,11.0,NaN,NaN,"DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRA..."
2,111,1,11.0,111.0,NaN,MEMBROS SUPERIORES DO PODER EXECUTIVO E LEGISL...
3,1111,1,11.0,111.0,1111.0,LEGISLADORES
4,1112,1,11.0,111.0,1112.0,DIRIGENTES SUPERIORES DA ADMINISTRAÇÃO PÚBLICA
...,...,...,...,...,...,...
613,0511,0,5.0,51.0,511.0,OFICIAIS DE BOMBEIRO MILITAR
614,0512,0,5.0,51.0,512.0,GRADUADOS E PRAÇAS DO CORPO DE BOMBEIROS
615,0599,0,5.0,59.0,599.0,BOMBEIROS E POLICIAIS MILITARES NÃO ESPECIFICADOS
616,0999,0,9.0,99.0,999.0,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."


In [75]:
ocupacoes_agregado = pd.merge(onet_soc, cod2010, how='left', left_on='ISCO-08 Code', right_on='Cod')

#### trabalhos teletrabalháveis

In [25]:
ocupacoes_agregado = isco_cod[['code_ONET', 'Title_ONET', 'teleworkable', 'Cod', 'Titulação']]

In [27]:
ocupacoes_agregado['Titulação'].value_counts().sort_values(ascending=False) 

Titulação
PROFISSIONAIS DAS CIÊNCIAS E INTELECTUAIS                                                               273
TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO                                                                 265
OUTROS                                                                                                  168
TRABALHADORES QUALIFICADOS, OPERÁRIOS E ARTESÃOS DA CONSTRUÇÃO, DAS ARTES MECÂNICAS E OUTROS OFÍCIOS    163
DIRETORES E GERENTES                                                                                    126
OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES                                                       104
TRABALHADORES QUALIFICADOS DA AGROPECUÁRIA, FLORESTAIS, DA CAÇA E DA PESCA                               79
TRABALHADORES DOS SERVIÇOS, VENDEDORES DOS COMÉRCIOS E MERCADOS                                          68
OCUPAÇÕES ELEMENTARES                                                                                    56
TRABALHADORES DE A

In [29]:
ocupacoes_agregado['Cod']

,code_ONET,Title_ONET,teleworkable,Cod,Titulação
0,11-1011,Chief Executives,1,1,DIRETORES E GERENTES
1,11-1011,Chief Executives,1,1,DIRETORES E GERENTES
2,11-1011,Chief Executives,1,1,DIRETORES E GERENTES
3,11-1011,Chief Sustainability Officers,1,1,DIRETORES E GERENTES
4,11-1011,Chief Sustainability Officers,1,1,DIRETORES E GERENTES
...,...,...,...,...,...
1334,53-7073,Wellhead Pumpers,0,8,OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES
1335,53-7081,Refuse and Recyclable Material Collectors,0,9,OCUPAÇÕES ELEMENTARES
1336,53-7081,Refuse and Recyclable Material Collectors,0,9,OCUPAÇÕES ELEMENTARES
1337,53-7111,Mine Shuttle Car Operators,0,8,OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES


In [80]:
ocupacoes_agregado.to_excel('../Anexos/ocupacoes_traduzidas.xlsx', index=False)